**Задание 1**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests

In [2]:
keywords = ['python', 'парсинг', 'решения']

In [3]:
import time

In [4]:
import pandas as pd

In [5]:
URL = 'https://habr.com/ru/all/'

In [6]:
from bs4 import BeautifulSoup

In [7]:
pages = 3

result_list = []

for i in range(pages + 1):
    if i == 0:
        URL = 'https://habr.com/ru/all/'
    else:
        URL = 'https://habr.com/ru/all/' + 'page' + str(i) +'/' 
    req = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    news_blocks = soup.find_all('article', class_='tm-articles-list__item')
    articles_date = list(map(lambda x: x.find('span', class_='tm-article-snippet__datetime-published').find('time').get('title'), news_blocks))
    articles_title = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title').find('span').get_text(), news_blocks))
    articles_text = list(map(lambda x: x.find('div', class_='article-formatted-body').get_text(), news_blocks))
    articles_link = list(map(lambda x: x.find('a', class_='tm-article-snippet__title-link').get('href'), news_blocks))
    result_list = result_list + list(zip(articles_date, articles_title, articles_text, articles_link))

result_list

[('2021-07-12, 19:43',
  'Security Week 28: уязвимости PrintNightmare в деталях',
  'На прошлой неделе специалисты «Лаборатории Касперского» опубликовали разбор двух уязвимостей в системе печати Windows, получившие название PrintNightmare. В среду 7 июля уязвимости закрыли патчем для Windows 7, 10 и серверных ОС начиная с Windows Server 2008 SP2. \n\n\r\nПроблемы в службе печати Windows, приводящие к выполнению произвольного кода, представляют интерес по двум причинам. Во-первых, это очень похоже на более раннюю уязвимость в Windows Print Spooler, использованную в атаке Stuxnet. Во-вторых, первооткрыватели бага случайно выложили PoC-код на GitHub еще в середине июня, когда вендор даже не анонсировал активно эксплуатируемую дыру в Windows.',
  '/ru/company/kaspersky/blog/567376/'),
 ('2021-07-12, 19:34',
  'Внеднение In-app Updates в Android-приложение на React Native',
  'В 2019 году Google выпустила\xa0In-app Updates\xa0— возможность обновлять Android-приложения без перехода в Google 

In [8]:
news = pd.DataFrame()

for e in keywords:
    for el in result_list:
        if e in  el[1].lower() or e in el[2].lower():
            row = {'date': el[0], 'title': el[1], 'link': 'https://habr.com' + el[3]}
            news = pd.concat([news, pd.DataFrame([row])]) 

In [9]:
news

,date,title,link
0,"2021-07-11, 19:31",Модели глубоких нейронных сетей sequence-to-se...,https://habr.com/ru/post/567142/
0,"2021-07-12, 17:49",Путь Namex IXP к IP-фабрикам,https://habr.com/ru/company/otus/blog/567320/
0,"2021-07-12, 15:23",Можно ли сэкономить на СХД?,https://habr.com/ru/company/nubes/blog/567306/
0,"2021-07-12, 17:49",Путь Namex IXP к IP-фабрикам,https://habr.com/ru/company/otus/blog/567320/
0,"2021-07-12, 15:23",Можно ли сэкономить на СХД?,https://habr.com/ru/company/nubes/blog/567306/
0,"2021-07-12, 13:08",Особенности создания синтаксического анализато...,https://habr.com/ru/post/567268/
0,"2021-07-12, 12:26",Оптимизация нагрузки на сеть с помощью разност...,https://habr.com/ru/company/fgts/blog/567098/
0,"2021-07-12, 08:02",Методы укладки Gephi. Force Atlas и Force Atlas 2,https://habr.com/ru/post/567198/


**Задание 2**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [10]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [11]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [12]:
res = requests.post(url, headers = {'Vaar-Version': '0', 
                                    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
                                    'Vaar-Header-App-Build-Version': '1.0.0'
                                   }, 
                    json = {'emailAddresses': EMAIL})
res.text

'null'

In [13]:
res.json()

In [14]:
df = pd.DataFrame(res.json()['breaches'])
df

TypeError: 'NoneType' object is not subscriptable

Раньше всё это работало, но уже полвечера не могу получить никакого результата. Может бан? Дальше я хотел с помощью pd.pivot_table(df, values = 'Value', index=['','',''], columns = '').reset_index() развернуть таблицу и оставить только три столбца